# Contest with embedding-based models

In [2]:
from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from datasets import load_dataset
from sklearn.metrics import precision_recall_fscore_support
import pandas as pd

c:\Users\ltoma\anaconda3\envs\NLP\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## IMDB Dataset

In [3]:
ds = load_dataset("stanfordnlp/imdb")
train = ds["train"]
test = ds["test"]

In [4]:
results_list = []

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    return {'precision': precision, 'recall': recall, 'f1': f1}

## Fine-tune approach

### TinyBERT

In [ ]:
model_name = "huawei-noah/TinyBERT_General_4L_312D"

training_args = TrainingArguments(
    num_train_epochs=2,
    output_dir="./tinybert_results",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding='max_length', truncation=True, max_length=512)

train = train.map(tokenize_function, batched=True)
test = test.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer)


trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
    compute_metrics=compute_metrics
)

trainer.train()
results = trainer.evaluate()

trainer.save_model("./tinybert_model")
tokenizer.save_pretrained("./tinybert_model")

results_list.append({
    "Model": model_name,
    "Precision": results["eval_precision"],
    "Recall": results["eval_recall"],
    "F1 Score": results["eval_f1"]
})

### DistilBERT

In [ ]:
model_name = "distilbert-base-uncased"

training_args = TrainingArguments(
    num_train_epochs=1,
    output_dir="./distilbert_results",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True)

train = train.map(tokenize_function, batched=True)
test = test.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer)


trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
    compute_metrics=compute_metrics
)

trainer.train()
results = trainer.evaluate()

trainer.save_model("./distilbert_model")
tokenizer.save_pretrained("./distilbert_model")

results_list.append({
    "Model": model_name,
    "Precision": results["eval_precision"],
    "Recall": results["eval_recall"],
    "F1 Score": results["eval_f1"]
})

## Pretrained models

## Pretrained distillbert (3 epochs)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("derek-harnett/movie-review-classifier")
model = AutoModelForSequenceClassification.from_pretrained("derek-harnett/movie-review-classifier")

training_args = TrainingArguments(
    output_dir="./distilbert_pretrained_results",
    per_device_eval_batch_size=16,
    evaluation_strategy="no"
)

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True)

test = test.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer)


trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
    args=training_args,
    eval_dataset=test,
    compute_metrics=compute_metrics
)

results = trainer.evaluate()

results_list.append({
    "Model": "Movie Review Classifier - Pretrained",
    "Precision": results["eval_precision"],
    "Recall": results["eval_recall"],
    "F1 Score": results["eval_f1"]
})

## Results

In [5]:
# add scores obtained on colab
results_list.append({
    'Model': 'huawei-noah/TinyBERT_General_4L_312D', 
    'Precision': 0.8984355803096077, 
    'Recall': 0.87752, 
    'F1 Score': 0.8878546278683881
})

results_list.append({
    'Model': 'distilbert-base-uncased', 
    'Precision': 0.9362309385957759, 
    'Recall': 0.91848, 
    'F1 Score': 0.9272705245729516
})



results_list.append({
    'Model': 'Movie Review Classifier - Pretrained', 
    'Precision': 0.9273287996204334, 
    'Recall': 0.93816, 
    'F1 Score': 0.9327129563350036
})

results_df = pd.DataFrame(results_list)



In [6]:
print(results_df)

                                  Model  Precision   Recall  F1 Score
0  huawei-noah/TinyBERT_General_4L_312D   0.898436  0.87752  0.887855
1               distilbert-base-uncased   0.936231  0.91848  0.927271
2  Movie Review Classifier - Pretrained   0.927329  0.93816  0.932713
